# 台股及時看板

最後我們就發揮我們在過去所學，利用 **Excel、Python**、以及股價 **fugle Web API** 來打造一個及台股即時看板： 

完成品影片：https://youtu.be/Gd_bQTRUrzQ

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/Gd_bQTRUrzQ/0.jpg)](https://www.youtube.com/watch?v=Gd_bQTRUrzQ)


## 目標

該即時看板應該具備以下功能：

1. 顯示即時股價
2. 顯示即時股價的走勢圖
3. 允許使用者選擇不同的股票
4. 能夠判斷交易條件並且即時通知使用者的功能


In [ ]:
import xlwings as xw

wb = xw.Book(r"dashboard.xlsx")
wb

## 1. 顯示即時的股價

fugle 即時股價 API 教學請參考：https://developer.fugle.tw/realtime/document

![](https://drive.google.com/uc?export=download&id=161WgYDildws9AJMNaT8Oha0VdJXmYUy_)

In [ ]:
# 將 portfolio 工作表存入 port_sheet
watch_list = wb.sheets["觀察清單"]
watch_list

In [ ]:
# 截取玉山個股的基本資訊
import requests

payload = {
    "symbolId": "2884",
    "apiToken": "demo"
}

res = requests.get("https://api.fugle.tw/realtime/v0/intraday/meta", params=payload)
res.json()

# 但是我們需要把截取到的資料寫入到工作表内

回顧一下第一課的教材，若要一次將多筆資料寫入到一個工作表的範圍内，我們可以先將所有資料都封裝到一個串列内

我們可以先觀察需要寫入的範圍的大小：

![](https://drive.google.com/uc?export=download&id=14V9m9pqMuAbcTfkdKWuuCSTSRku5IFRF)

**B2:I2** 是一個 1 x 8 大小的範圍

在這樣的狀況下，我們必須透過 Python 把資料變成以下格式：

```python
['玉山金', 27.15, 27.25, 27.15, 27.25, 27.05]
```

開高低收的資料都不難取得，但是股票中文簡稱和昨日收盤價呢？

這時我們就可以把 `fulge api` 的 `meta` 拿出來使用 

In [ ]:
import requests

payload = {
    "symbolId": "2884",
    "apiToken": "demo"
}

res = requests.get("https://api.fugle.tw/realtime/v0/intraday/meta", params=payload)

json_data = res.json()
data = json_data["data"]["meta"]

data["nameZhTw"]

# 改寫一下我們之前打造的 fugle_stock_crawler

```python
def fugle_stock_crawler(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote", params=payload)
    json_data = res.json()

    res1 = requests.get("https://api.fugle.tw/realtime/v0/intraday/meta", params=payload)
    json_data1 = res1.json()

    return {
        "open": json_data["data"]["quote"]["priceOpen"]["price"],
        "high": json_data["data"]["quote"]["priceHigh"]["price"],
        "low": json_data["data"]["quote"]["priceLow"]["price"],
        "close": json_data["data"]["quote"]["trade"]["price"],
        "lastClose": json_data1["data"]["meta"]["priceReference"],
        "name": json_data1["data"]["meta"]["nameZhTw"]
    }
```

In [ ]:
def fugle_stock_crawler(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote", params=payload)
    json_data = res.json()

    res1 = requests.get("https://api.fugle.tw/realtime/v0/intraday/meta", params=payload)
    json_data1 = res1.json()

    return {
        "open": json_data["data"]["quote"]["priceOpen"]["price"],
        "high": json_data["data"]["quote"]["priceHigh"]["price"],
        "low": json_data["data"]["quote"]["priceLow"]["price"],
        "close": json_data["data"]["quote"]["trade"]["price"],
        "lastClose": json_data1["data"]["meta"]["priceReference"],
        "name": json_data1["data"]["meta"]["nameZhTw"]
    }

In [ ]:
fugle_stock_crawler("2884", "demo")

# 將我們剛才處理資料的流程封裝到函數内

一旦在修改同行碼之後，我們的程式就可以截取多支不同的股票資訊了

In [ ]:
api_token = watch_list.range("L1").value
stock_id = "2884"

print(api_token)
fugle_stock_crawler(stock_id, api_token)

In [ ]:
api_token = watch_list.range("L1").value
stock_id = "2884"

data = fugle_stock_crawler(stock_id, api_token)


[
    data["name"],
    data["open"],
    data["high"],
    data["low"],
    data["close"],
    data["lastClose"]
]

In [ ]:
api_token = port_sheet.range("L1").value
stock_id = "2884"

data = fugle_stock_crawler(stock_id, api_token)

watch_list.range("B2").value = [data["name"],data["open"],data["high"],data["low"],data["close"],data["lastClose"]]

# 我們就可以成功將資料寫入工作表了
![](https://drive.google.com/uc?export=download&id=161WgYDildws9AJMNaT8Oha0VdJXmYUy_)

最後只需要輸入一個公式，我們就可以將漲跌幅度計算出來：

![](https://drive.google.com/uc?export=download&id=18mW3ehmeHvaW44QFWFxyYxR10VecXf6J)

# 動態截取使用者輸入的觀察清單

若要頻繁的修改程式碼，並不是一個聰明的做法，因爲會增加改錯的風險

所以我們就把程式改成：

1. 動態截取使用者輸入的股票代號
2. 動態截取資料
3. 寫入工作表

In [ ]:
last_row = watch_list.range("A1").end("down").row

for i in range(2, last_row+1):
    stock_id = watch_list.range(f"A{i}").value
    print(stock_id)

In [ ]:
last_row = watch_list.range("A1").end("down").row

for i in range(2, last_row+1):
    stock_id = watch_list.range(f"A{i}").value
    stock_id = str(int(stock_id)) if type(stock_id) == float else str(stock_id)
    print(stock_id)

In [ ]:
# 偵測最後一行
last_row = watch_list.range("A1").end("down").row

# 從第二行到最後一行
for i in range(2, last_row+1):
    # 截取該行 A 欄的資料
    stock_id = watch_list.range(f"A{i}").value
    stock_id = str(int(stock_id)) if type(stock_id) == float else str(stock_id)
    print(stock_id)
    # 透過 fugle api 截取該股票資料
    data = fugle_stock_crawler(stock_id, api_token)
    print(data)

In [ ]:
# 偵測最後一行
last_row = watch_list.range("A1").end("down").row

# 從第二行到最後一行
for i in range(2, last_row+1):
    # 截取該行 A 欄的資料
    stock_id = watch_list.range(f"A{i}").value
    stock_id = str(int(stock_id)) if type(stock_id) == float else str(stock_id)
    print(stock_id)
    data = fugle_stock_crawler(stock_id, api_token)
    # 將結果寫入觀察清單的同一行
    watch_list.range(f"B{i}").value = [data["name"],data["open"],data["high"],data["low"],data["close"],data["lastClose"]]

最後我們的觀察清單就會顯示：

![](https://drive.google.com/uc?export=download&id=1clkVXuWuj9tOWk8XXCpogJRV7_q8d6Z4)

## 2. 具備顯示即時的走勢圖

我們接下來要打造一個即時走勢圖的功能

用 Python 產生走勢圖的實作方法有很多種，本課程將會使用 Excel 内建的圖表功能來實作走勢圖

Excel 圖表的類型衆多，繪製走勢圖我們會使用**開盤-高-低-收盤股價圖**：

![](https://www.dropbox.com/s/8q8n9tpzn118fl4/ohlc_chart.PNG?dl=1)

繪製圖表前，最麻煩的事情是我們需要一個一個交易日的資料，並且要把資料整理成以下格式：

![](https://www.dropbox.com/s/qo2m4phyo7sk9if/real_time_chart_data.PNG?dl=1)

此時我們就可以使用 fugle 的 `/intraday/chart` api 來截取需要的資料

In [ ]:
import requests

payload = {
    "symbolId": "2884",
    "apiToken": api_token
}

res = requests.get("https://api.fugle.tw/realtime/v0/intraday/chart", params=payload)

result = res.json()
result

# 觀察一下回傳的結果

```json
{
  {'2019-06-28T01:01:00.000Z': {'close': 26.3,
    'high': 26.3,
    'low': 26.3,
    'open': 26.3,
    'unit': 426,
    'volume': 426000},
   '2019-06-28T01:02:00.000Z': {'close': 26.3,
    'high': 26.3,
    'low': 26.3,
    'open': 26.3,
    'unit': 47,
    'volume': 47000}
   }
}
```

我們要的資料是被放在 data > chart 内，**每一個交易時段與相對應的開盤-高-低-收盤股價是以 key : value 的形式被封裝在字典内**

但是我們需要把這些資料轉成二維陣列，才有可能將其批次的寫入 Excel 工作表：


```python
[
    ['2019-06-28T01:01:00.000Z', '26.3', '26.3', '26.3', '26.3','426', '426000'],
    ['2019-06-28T01:02:00.000Z', '26.3', '26.3', '26.3', '26.3', '47', '47000'],
    ['2019-06-28T01:03:00.000Z', '26.3', '26.35', '26.3', '26.35', '88', '88000']
]
```

In [ ]:
# 我們先將所有時段的時間戳讀取成一個串列
time_idx = result["data"]["chart"].keys()
time_idx

In [ ]:
# 將單個時段的
price_open = []

data = result["data"]["chart"]

for time in time_idx:
    price_open.append(data[time]["open"])

price_open

In [ ]:
price_open = []
price_high = []
price_low = []
price_close = []
unit = []
volume = []
data = result["data"]["chart"]

for time in time_idx:
    price_open.append(data[time]["open"])
    price_high.append(data[time]["high"])
    price_low.append(data[time]["low"])
    price_close.append(data[time]["close"])
    unit.append(data[time]["unit"])
    volume.append(data[time]["volume"])
    
    
[price_open,price_high,price_low,price_close,unit,volume]

In [ ]:
# 將 TW884 存入 target_sheet
target_sheet = wb.sheets["TW2884"]

# 將二維串列的資料指定給 A2 為左上角的範圍
target_sheet.range("A2").value = [list(time_idx), price_open,price_high,price_low,price_close,unit,volume]

# 但是結果卻出乎我們意料之外

![](https://www.dropbox.com/s/kq8f8fa4svvemtg/chart_b4_transpose.PNG?dl=1)

原因是我們的二維串列的維度不對，**今天要將資料寫入一個二維的 Excel 工作表範圍，就需要賦值一個維度大小一樣的二維串列**

我們現在需要把二維串列做一個**轉置（Transpose）的處理，利用 Numpy 套件可以很快完成**

In [ ]:
import numpy as np

arr = np.array([list(time_idx), price_open,price_high,price_low,price_close,unit,volume]) 
arr.shape

In [ ]:
arr = np.array([list(time_idx), price_open,price_high,price_low,price_close,unit,volume]) 
arr.T

In [ ]:
target_sheet.range("A2").value = arr.T

In [ ]:
import requests
import numpy as np

def get_chart_data(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/chart", params=payload)

    result = res.json()

    price_open = []
    price_high = []
    price_low = []
    price_close = []
    unit = []
    volume = []

    time_idx = result["data"]["chart"].keys()

    for time in time_idx:
        data = result["data"]["chart"]
        price_open.append(data[time]["open"])
        price_high.append(data[time]["high"])
        price_low.append(data[time]["low"])
        price_close.append(data[time]["close"])
        unit.append(data[time]["unit"])
        volume.append(data[time]["volume"])

    real_time_data = [list(time_idx), price_open,price_high,price_low,price_close,unit,volume]
    stock_data_ary = np.array(real_time_data)
    
    return stock_data_ary.T

In [ ]:
data = get_chart_data(2884, "demo")
print(data)
print(data.shape)

In [ ]:
# 將 TW2884 存入 target_sheet
target_sheet = wb.sheets["走勢資料"]
# 將二維串列的資料指定給 A2 為左上角的範圍
target_sheet.range("A2").value = get_chart_data(2884, "demo")

# 用 Python 操作 Excel 圖表

In [ ]:
dashboard = wb.sheets["即時看板"]
# 將即時看板内的圖表物件都讀取出來
dashboard.charts

In [ ]:
# 選擇即時看板内名爲 chart1 的圖表，存入 chart 變數
chart = dashboard.charts["chart1"]
chart

In [ ]:
dashboard = wb.sheets["即時看板"]
target_sheet = wb.sheets["走勢資料"]

chart = dashboard.charts["chart1"]

last_cell = target_sheet.range("E1").end("down")
# 選擇 走勢資料 上，B2 到 E267 這個範圍，
data = target_sheet.range(f"B2:E{last_cell.row}")
# 將圖表的資料設定成該範圍内的資料
chart.set_source_data(data)
# 最後將圖表的類別設定成 開盤-高-低-收盤股價圖
chart.chart_type = "stock_ohlc"

# 3. 允許使用者選擇不同的股票 

In [ ]:
dashboard = wb.sheets["即時看板"]
# 截取使用者選擇的股票代號
stock_id = dashboard.range("stock1").value

target_sheet = wb.sheets["走勢資料"]

# avoid ETF stock number that begins with `0`
if type(stock_id) == float:
    stock_id = str(int(stock_id))

# 清理工作表的舊資料
target_sheet.range("A2").expand().clear_contents()
# 寫入資料
target_sheet.range("A2").value = get_chart_data(stock_id, "demo")

chart = dashboard.charts["chart1"]
last_cell = target_sheet.range("E1").end("down")
chart.set_source_data(target_sheet.range(f"B2:E{last_cell.row}"))
chart.chart_type = "stock_ohlc"

In [ ]:
import time

dashboard = wb.sheets["即時看板"]

while True:
    # 截取即時看板上的股票代號
    stock_id = dashboard.range("stock1").value

    # 將被讀取成浮點數的股票代號轉成字串
    if type(stock_id) == float:
        stock_id = str(int(stock_id))
        
    target_sheet = wb.sheets["走勢資料"]

    # 呼叫函數，將截取到的圖表資料（一個二維陣列）寫入以 A2 為起點的範圍
    target_sheet.range("A2").value = get_chart_data(stock_id, "demo")

    chart = dashboard.charts["chart1"]
    last_cell = target_sheet.range("E1").end("down")
    chart.set_source_data(target_sheet.range(f"B2:E{last_cell.row}"))
    chart.chart_type = "stock_ohlc"
    # 將最後一筆收盤價寫入看板工作表
    dashboard.range("price1").value = last_cell.value
    
    time.sleep(10)


# 4. 即時截取使用者在工作表上輸入的到價資訊

In [ ]:
condition = dashboard.range("condition1").value
limit = dashboard.range("limit1").value
price = dashboard.range("price1").value
stock_id = int(dashboard.range("stock1").value)

print(condition)
print(limit)
print(price)
print(stock_id)

In [ ]:
# 寫出條件判斷
def check_condition(stock_id, price, condition, limit):
    msg = ""
    if condition == "<":
        if price < limit:
             msg += "{} 的價格低於 {}\n".format(stock_id, limit)
    elif condition == ">":
        if price > limit:
             msg += "{} 的價格高於 {}\n".format(stock_id, limit)
    elif condition == "=":
        if price == limit:
             msg += "{} 的價格等於 {}\n".format(stock_id, limit)
    
    return msg

In [ ]:
condition = dashboard.range("condition1").value
limit = dashboard.range("limit1").value
price = dashboard.range("price1").value
stock_id = dashboard.range("stock1").value

check_condition(stock_id, price, condition, limit)

In [ ]:
def line_notify(msg, line_token):
    line_url = "https://notify-api.line.me/api/notify"
    token = line_token
  
    headers = {
            "Authorization": "Bearer " + token, 
            "Content-Type" : "application/x-www-form-urlencoded"
        }

    payload = {'message': msg}
    r = requests.post(line_url, headers = headers, params = payload)
    return r

In [ ]:
import time

dashboard = wb.sheets["即時看板"]

fugle_token = wb.sheets["觀察清單"].range("L1").value
line_token = wb.sheets["觀察清單"].range("L3").value

while True:
    line_report = "歡樂碼農及時股價看板：\n\n"
    # 截取即時看板上的股票代號
    stock_id = dashboard.range("stock1").value

    # 將被讀取成浮點數的股票代號轉成字串
    if type(stock_id) == float:
        stock_id = str(int(stock_id))
        
    target_sheet = wb.sheets["走勢資料"]

    # 呼叫函數，將截取到的圖表資料（一個二維陣列）寫入以 A2 為起點的範圍
    target_sheet.range("A2").value = get_chart_data(stock_id, fugle_token)
    # 選擇即時看板的圖表物件
    chart = dashboard.charts["chart1"]
    last_cell = target_sheet.range("E1").end("down")
    # 將圖表物件的資料來源設定成範圍 B2:GN
    chart.set_source_data(target_sheet.range(f"B2:E{last_cell.row}"))
    # 將圖表類型設定成 ohlc（開盤-高-低-收盤股價圖）
    chart.chart_type = "stock_ohlc"
    # 將最後一筆收盤價寫入看板工作表
    dashboard.range("price1").value = last_cell.value
    # 讀取觸發 Line 通知的比較條件
    condition = dashboard.range("condition1").value
    limit = dashboard.range("limit1").value
    price = dashboard.range("price1").value
    # 檢查是否符合觸發 Line 通知，產生 Line 訊息的字串
    line_report += check_condition(stock_id, price, condition, limit)

    # 若 Line 訊息的字串與初始化的不符合，意味條件被觸發
    if line_report != "歡樂碼農即時股價看板：\n\n":
        # 發送 Line 訊息
        line_notify(line_report, line_token)

    time.sleep(10)